<a href="https://colab.research.google.com/github/Anilesh05/Anilesh/blob/main/Export_HDFS_To_Mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Installing jdk and hadoop***

In [ ]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-2.10.2/hadoop-2.10.2.tar.gz
!tar xf hadoop-2.10.2.tar.gz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-2.10.2"
os.environ["HADOOP_COMMON_HOME"] = "/content/hadoop-2.10.2"
os.environ["HADOOP_MAPRED_HOME"] = "/content/hadoop-2.10.2"
os.environ["PATH"] = os.environ["HADOOP_HOME"] + "/bin:" + os.environ["PATH"]


# ***Installing sqoop***

In [ ]:
!wget https://archive.apache.org/dist/sqoop/1.4.6/sqoop-1.4.6.bin__hadoop-2.0.4-alpha.tar.gz
!tar -xvf sqoop-1.4.6.bin__hadoop-2.0.4-alpha.tar.gz

import os
os.environ["SQOOP_HOME"] = "/content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha"
os.environ["PATH"] = os.environ["SQOOP_HOME"] + "/bin:" + os.environ["PATH"]

!cp /content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha/conf/sqoop-env-template.sh /content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha/conf/sqoop-env.sh
!echo "export HADOOP_COMMON_HOME=/content/hadoop-2.10.2" >> /content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha/conf/sqoop-env.sh
!echo "export HADOOP_MAPRED_HOME=/content/hadoop-2.10.2" >> /content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha/sqoop-env.sh

# ***Installing Mysql-Connector***

In [ ]:
!wget https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-8.3.0.tar.gz
!tar xf mysql-connector-j-8.3.0.tar.gz
!cp /content/mysql-connector-j-8.3.0/mysql-connector-j-8.3.0.jar /content/sqoop-1.4.6.bin__hadoop-2.0.4-alpha/lib

# ***Installing Mysql Server***


In [ ]:
!apt-get -y install mysql-server
!service mysql start
!pip install mysql-connector-python
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

In [ ]:
!java -version
!hadoop version
!mysql --version
!sqoop version

In [ ]:
!hdfs dfs -mkdir input

In [ ]:
import mysql.connector
from tabulate import tabulate

conn = mysql.connector.connect(user='root', password='root', host='localhost', port='3306')

cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Mysql_DB")

cursor.execute("USE Mysql_DB")

create_table_query = '''CREATE TABLE EMPLOYEE (
   ID INT,
   FIRST_NAME CHAR(20) NOT NULL,
   LAST_NAME CHAR(20),
   DEPT CHAR(20),
   YEAR INT,
   EMAIL CHAR(50)
)'''
cursor.execute(create_table_query)

conn.commit()

cursor.execute("SELECT * FROM EMPLOYEE")

result = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

print(tabulate(result, headers=columns, tablefmt="grid"))


cursor.close()
conn.close()

In [ ]:
%%writefile database.txt
1,Anilesh,C,BCA(DS),2,anil@gmail.com
2,Anilesh,C,BCA(DS),2,anil@gmail.com
3,Anilesh,C,BCA(DS),2,anil@gmail.com

In [ ]:
!cat /content/database.txt

In [ ]:
!hdfs dfs -mv /content/database.txt  input/export_table

In [ ]:
!hdfs dfs -cat input/export_table/

In [ ]:
!sqoop export \
  --connect jdbc:mysql://localhost:3306/Mysql_DB \
  --username root \
  --password root \
  --table EMPLOYEE \
  --export-dir input/export_table \
  --m 1

In [ ]:
conn = mysql.connector.connect(user='root', password='root', host='localhost', port='3306', database='Mysql_DB')

cursor = conn.cursor()

cursor.execute("SELECT * FROM EMPLOYEE")

result = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

print(tabulate(result, headers=columns, tablefmt="grid"))